<a href="https://colab.research.google.com/github/jborders23/math6397-funhouse/blob/main/MATH6397_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Processing

Import & Pre-Process Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load data from GitHub
tesla = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/TSLA.csv")
ford = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/F.csv")
gm = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/GM.csv")
sp500 = pd.read_excel("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/SP500.xlsx")
#reverse sp500 and reset index
sp500 = sp500.iloc[::-1].reset_index(drop=True)


In [2]:
#keep date and closing price for each security
t = tesla.iloc[:,[0,4,7]] #keep 'tomorrow' price for tesla, the target stock
f = ford.iloc[:,[0,4]]
g = gm.iloc[:,[0,4]]
s = sp500.iloc[:,[0,4]]

#add column to tesla that equals 1 if tomorrow>close
t['Movement'] = np.where(t['Tomorrow'] > t['Close'],1,0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [3]:
#concatenate dfs
data = pd.concat(objs=[f['Date'],s['Close'],f['Close'],g['Close'],t['Close'],t['Tomorrow'],t['Movement']],axis=1,keys=['Date','SP','Ford','GM','Tesla','Tomorrow','Movement'])
#normalize the time series: xi - min(xi)/max(xi) - min(xi)
s_data = data #will work with a standardized data frame
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
s_data[['SP','Ford','GM','Tesla','Tomorrow']] = scaler.fit_transform(s_data[['SP','Ford','GM','Tesla','Tomorrow']])
s_data.head()

,Date,SP,Ford,GM,Tesla,Tomorrow,Movement
0,4/26/2016,0.041722,0.966270,0.340939,0.017649,0.017113,0
1,4/27/2016,0.043301,0.957341,0.340275,0.017113,0.016226,0
2,4/28/2016,0.034450,1.000000,0.346478,0.016226,0.014585,0
3,4/29/2016,0.029639,0.947421,0.332300,0.014585,0.014831,1
4,5/2/2016,0.037022,0.953373,0.331192,0.014831,0.012593,0


# Convolution

https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

https://github.com/dhwanirc/Image-Classification-with-CNN-RF

Prepare Data for Convolution

In [54]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [64]:
# split s_data into what the network needs
d_data = s_data[['SP','Ford','GM','Tesla','Movement']]
X = d_data.iloc[:,0:4]
y = d_data.iloc[:,-1]
#convert X to array
X = np.array(X[:])
y = np.array(y[:])

In [ ]:
d_data['Movement'].mean() #Tesla's gone up 51.9% of trading days the past five years

In [65]:
#train/test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#reshape to 3d for convolution network
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1])

input_shape=(x_train.shape[0], x_train.shape[1]) #gets 4 cols
#should be correct dimension for convolution, flattening, mlp

batch_size = int(round(len(x_train)**0.5,0))
num_classes = 2
epochs = 100

Screw this, I'm just trying random forest as is.

In [74]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 200 trees
model = RandomForestClassifier(n_estimators=200, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [75]:
# Actual class predictions
rf_predictions = model.predict(x_test)
# Probabilities for each class
rf_probs = model.predict_proba(x_test)[:, 1]

In [80]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rf_probs) #pretty shit 0.574

Neural Network Cutoff

In [57]:
x_train = tf.convert_to_tensor(x_train).experimental_ref()
y_train = tf.convert_to_tensor(y_train).experimental_ref()
x_test = tf.convert_to_tensor(x_test).experimental_ref()
y_test = tf.convert_to_tensor(y_test).experimental_ref()

In [58]:
input_shape

(1005, 4)

In [59]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

#build
model = models.Sequential()
model.add(layers.Conv1D(filters=16,kernel_size=5,padding='same',activation='relu',input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=16,kernel_size=5,padding='same',activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2,activation='softmax'))
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 1005, 16)          336       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 502, 16)           1296      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 251, 16)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 4016)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)                200850    
_________________________________________________________________
dense_13 (Dense)             (None, 2)                

In [60]:
#fit model to get optimized CNN parameters
checkpointer = ModelCheckpoint('does mlp work?', monitor='val_accuracy', save_best_only=True)
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
monitor = model.fit(x_train, tf.one_hot(pd.Series(y_train,dtype='category').astype('category').cat.codes,2), epochs=epochs, batch_size= batch_size,
                    validation_data=(x_test, tf.one_hot(pd.Series(y_test, dtype='category').astype('category').cat.codes,2)), callbacks=[checkpointer])


AttributeError: ignored

extract first dense layer's inputs, send to random forest